In [1]:
# !pip3 install -q -U transformers
!pip3 install -q -U bitsandbytes
!pip3 install -q -U peft
!pip3 install -q -U trl
!pip3 install -q -U accelerate

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

2024-04-09 09:45:22.181855: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-09 09:45:22.181966: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-09 09:45:22.438927: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
from datasets import Dataset

In [4]:
from huggingface_hub import login
login()

In [5]:
import pandas as pd
import numpy as np

In [6]:
# train data preprocessing

DATA_PATH = '/kaggle/input/dobaehaza/'
train = pd.read_csv(f'{DATA_PATH}train.csv')

answer = ['답변_1', '답변_2', '답변_3', '답변_4', '답변_5']
question = ['질문_1', '질문_2']
bag = []
for i in range(len(train)):
    row = train.iloc[i,:]
    bag.extend([{'id':row['id'], 'question':row[j], 'answer':row[i]} for j in question for i in answer])

train = pd.DataFrame(bag)
train.sample(5)

,id,question,answer
682,TRAIN_068,포세린 타일의 단점이 뭐야,"포세린 타일은 시공 방법에 따라 하자 발생률이 높을 수 있고, 또한 오염에 취약하다..."
5500,TRAIN_550,"반려동물과 함께 살 때, 바닥에서 1미터 높이에 몰딩을 설치하는 것을 추천하는 이유...",바닥에서 1미터 높이에 몰딩을 설치하면 하단부의 벽지가 훼손될 때마다 하단부의 벽지...
2665,TRAIN_266,워쉬프라이머가 무엇인지 알려주세요.,워쉬프라이머는 합성수지를 전색제로 하여 안료와 인산을 첨가한 것으로 주로 뿜칠로 도...
2801,TRAIN_280,흡음재 중 판 진동형이 뭐야?,"흡음재 중 판 진동형은 벽과 일정 간격을 두고 설치되는 얇고 통기성이 없는 판으로,..."
861,TRAIN_086,비드법 보온판 2종에 대해 설명해줘,"비드법 보온판 2종은 네오폴, 에너포르, 제로폴 등의 상표명으로도 알려져 있으며, ..."


In [7]:
# 질문을 기준으로 데이터 그룹화 후 가장 긴 질문 선택
train_g = train.groupby('question')['answer'].apply(lambda x: x.loc[x.str.len().idxmax()]).reset_index(name='answer')

train_g


,question,answer
0,2023년의 인기 있는 인테리어 디자인 트렌드는 어떤 것이 있을까요?,"2023년의 최신 인테리어 디자인 트렌드로는 천연 소재 활용, 어스톤 컬러 팔레트,..."
1,2023년의 최신 인테리어 디자인 트렌드가 무엇인가요?,"2023년의 최신 인테리어 디자인 트렌드로는 천연 소재 활용, 어스톤 컬러 팔레트,..."
2,ABC 소화기가 뭐야?,"ABC 소화기는 일반적으로 가장 많이 볼 수 있는 소화기로 A, B, C 등급의 화..."
3,ABC 소화기란 무엇인가요?,"ABC 소화기는 일반적으로 가장 많이 볼 수 있는 소화기로 A, B, C 등급의 화..."
4,"AD, PD에 면한 벽체 결로에 대한 대책은 뭐야?","AD, PD에 면한 벽체 결로에 대한 대책은 단열재를 미실하게 시공하여 결로가 생기..."
...,...,...
1281,흡음재 중 다공질이 뭐야?,"흡음재 중 다공질은 미세 구멍이나 틈이 많이 있는 소재를 말합니다. 유리면, 암면,..."
1282,흡음재 중 판 진동형이 뭐야?,판 진동형 흡음재는 간격을 두고 얇고 통기성이 없는 판을 설치하여 소리가 전달될 때...
1283,흡음재 중에 다공질이 무엇인가요?,"흡음재 중 다공질은 미세 구멍이나 틈이 많이 있는 소재를 말합니다. 유리면, 암면,..."
1284,흡음재 중에서 공명형은 무엇을 의미하는 건가요?,흡음재 중 공명형은 작은 구멍이나 틈을 이용하여 소리의 공명을 이용하는 재질을 말합...


In [8]:
train_data = Dataset.from_pandas(train_g)


In [11]:
def generate_prompt(example):
    prompt_list = []
    for i in range(len(example['question'])):
        prompt_list.append(r"""<bos><start_of_turn>user
        다음 글을 답변해주세요:

        {}<end_of_turn>
        <start_of_turn>model
        {}<end_of_turn><eos>""".format(example['question'][i], example['answer'][i]))
    return prompt_list

In [12]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,
    lora_alpha = 32,
    lora_dropout = 0.05,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    bias="none",
    task_type="CAUSAL_LM",
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [13]:
import torch
torch.cuda.empty_cache()
import gc

gc.collect()

885

In [14]:
BASE_MODEL = "google/gemma-2b-it"

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map={'':0}, quantization_config=bnb_config, )
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, add_special_tokens=True)
tokenizer.padding_side = 'right'

from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

In [15]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    max_seq_length=512,
    args=TrainingArguments(
        output_dir="outputs",
       num_train_epochs = 1,
        max_steps=1500,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=1,
        optim="paged_adamw_8bit",
        warmup_steps=0.03,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=100,
        push_to_hub=False,
        report_to='none',
    ),
    peft_config=lora_config,
    formatting_func=generate_prompt,
)

Map:   0%|          | 0/1286 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:317: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [16]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
100,1.825000
200,1.382300
300,1.293000
400,1.204000
500,1.103400
600,1.037900
700,0.765800
800,0.578500
900,0.575300
1000,0.514700


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain curr

TrainOutput(global_step=1500, training_loss=0.7910810165405273, metrics={'train_runtime': 5152.5243, 'train_samples_per_second': 0.582, 'train_steps_per_second': 0.291, 'total_flos': 8396934184796160.0, 'train_loss': 0.7910810165405273, 'epoch': 2.33})

In [17]:
ADAPTER_MODEL = "lora_adapter_it"

trainer.model.save_pretrained(ADAPTER_MODEL)

In [18]:
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, device_map='auto', torch_dtype=torch.float16)
model = PeftModel.from_pretrained(model, ADAPTER_MODEL, device_map='auto', torch_dtype=torch.float16)

model = model.merge_and_unload()
model.save_pretrained('gemma-ko-2b-it')
model.push_to_hub('doxgxxn/dobae_gemma')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/doxgxxn/dobae_gemma/commit/4ebacaab7489ca8280478345bb64236b3c1d416b', commit_message='Upload GemmaForCausalLM', commit_description='', oid='4ebacaab7489ca8280478345bb64236b3c1d416b', pr_url=None, pr_revision=None, pr_num=None)

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

class AIAssistant():
    def __init__(self, model_name="google/gemma-2b-it", finetune_model="doxgxxn/dobae_gemma", temperature=0.2, top_k=50, top_p=0.95):
        """Initialize the AI assistant."""

        # Initialize attributes
        self.finetune_model = AutoModelForCausalLM.from_pretrained(finetune_model, device_map={"":0})
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, add_special_tokens=True)
        self.pipe_finetuned = pipeline("text-generation", model=self.finetune_model, tokenizer=self.tokenizer, max_new_tokens=512)
        self.temperature = temperature
        self.top_k = top_k
        self.top_p = top_p
    
    def query(self, query):
        """Query the knowledge base of the AI assistant"""
        
        message = [
                     {
                        "role": "user",
                        "content": "다음 글을 답변해주세요:\n\n{}".format(query)
                    }
                  ]
        prompt = self.pipe_finetuned.tokenizer.apply_chat_template(message, tokenize=False, add_generation_prompt=True)
        outputs = self.pipe_finetuned(
                                        prompt,
                                        do_sample=True,
                                        temperature=self.temperature,
                                        top_k=self.top_k,
                                        top_p=self.top_p,
                                        add_special_tokens=True
            
                                        )
        return outputs[0]['generated_text'][len(prompt):]
    
    def set_temperature(self, temperature):
        self.temperature = temperature
        
    def set_top_k(self, top_k):
        self.top_k = top_k
        
    def set_top_p(self, top_p):
        self.top_p = top_p

In [10]:
gemma_ai_assistant = AIAssistant()

config.json:   0%|          | 0.00/691 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


...

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

In [12]:
pd.set_option('display.max_colwidth', None)
train.sample(10)

,id,question,answer
824,TRAIN_082,비드법 보온판 2종의 단점은 뭐야?,비드법 보온판 2종의 단점으로는 숙성기간이 필요하고 열에 취약하다는 점이 있습니다. 이러한 특성으로 인해 시공 및 유지보수 측면에서 추가적인 주의가 필요합니다.
1457,TRAIN_145,비드법 보온판 1종의 장점은 무엇인가요?,"비드법 보온판 1종의 단점은 다음과 같습니다. 1. 숙성 과정이 길어야 함: 비드법 보온판 1종은 7주 이상의 숙성 과정을 거쳐야 하기 때문에 시공 시간이 상대적으로 오래 걸릴 수 있습니다. 2. 수분에 민감함: 물에 노출되면 단열 성능이 저하될 수 있어, 공간 내 습기 관리에 더욱 신경 써야 합니다. 3. 화재에 취약: 비드법 보온판 1종은 화재에 취약하며, 화재 시에 발생하는 유독가스도 주의해야 합니다."
1238,TRAIN_123,쌀풀은 무엇이며 어떤 용도로 사용되나요?,"쌀풀은 쌀을 가공하여 제작된 재료로, 친환경적이고 접착력이 밀풀에 비해 약 1.5배 좋으며 내구성이 좋은 재료입니다. 이러한 장점을 갖고 있지만, 다소 비용이 비싸다는 단점도 있습니다. 쌀풀은 건물 내부 및 외부 벽면, 천장, 바닥 등 다양한 장소에 도배 재료로 사용되며, 좋은 품질을 유지하고자 하는 경우에 선택할 수 있는 재료입니다."
1691,TRAIN_169,석고보드를 사용하는 이유가 뭐야?,"석고보드는 건축자재로 많이 사용되는데, 그 이유는 무해하고 친환경적인 소재이기 때문입니다. 또한 가볍고 쉽게 절단할 수 있어 경제적이며, 단열성능이 뛰어나서 건물 내부의 온도와 소음을 효과적으로 조절할 수 있습니다. 따라서 인테리어 디자인이나 건축물의 내부 구조물로 많이 사용되는 재료 중 하나입니다."
4735,TRAIN_473,수도꼭지가 누수할 때 어떤 수리 방법을 사용해야 하나요?,수도꼭지 부분을 분해하여 밀봉 고무링을 교체하거나 파손된 부품을 교체하세요.
123,TRAIN_012,철골콘크리트 구조가 뭐야?,"철골콘크리트는 철근을 사용하여 콘크리트와 결합하여 건축물의 구조를 형성하는 구조물입니다. 기둥, 보, 내력벽, 바닥 슬라브 등의 주요 구조부가 철근 콘크리트로 시공되어 강한 인장력과 압축력을 견디는 구조를 형성합니다. 이러한 구조는 자유로운 형상을 구현하는 것이 가능하며, 내구성, 내화성, 내진성을 갖추고 있습니다. 하지만 비강도가 비교적 작고, 여러 가지 이유로 균열 발생 위험이 있는 점이 단점으로 지적되고 있습니다."
90,TRAIN_009,블록구조가 뭐야,블록구조는 조적식 구조법의 하나로 철근 콘크리트가 발달된 이후에 나타난 비교적 새로운 구조입니다. 모르타르 또는 콘크리트 블록을 만들어서 벽돌재와 같은 방법으로 벽체를 구축하는 구조이며 철근 콘크리트로 보강하여 사용하기도 합니다.
775,TRAIN_077,강화마루의 기대 수명은 얼마인가요?,강화마루의 기대수명은 25년입니다.
2094,TRAIN_209,성분에 따른 페인트 종류 좀 알려줘,"성분에 따른 페인트 종류로는 주로 우레탄 계열, 에폭시 계열, 에나멜 계열, 아크릴 계열 등이 있습니다. 이러한 종류의 페인트는 사용되는 장소나 목적에 따라 선택되며, 각각의 특성과 용도에 대해 더 자세히 알아보시는 것이 좋습니다."
543,TRAIN_054,방음이 뭐야,"방음은 건물 내부에서 발생하는 소음을 외부로 유출하지 않도록 막는 것을 말합니다. 방음은 주로 건축물의 외벽이나 바닥, 천장 등의 재료나 구조적 요소를 통해 소음이 퍼지는 것을 막는 역할을 합니다. 방음은 흡음과 차음 두 가지로 나누어집니다. 흡음은 소음의 에너지를 흡수해서 진행을 방해하는 것이고, 차음은 소음을 외부로 방출하지 않도록 막는 것을 의미합니다. 이를 통해 건물 내부의 안락한 환경을 조성하고 주변 환경에 불편을 끼치지 않도록 하는 역할을 합니다."


In [13]:
gemma_ai_assistant.query('비드법 보온판 2종의 단점은 뭐야?')

'비드법 보온판 2종의 또 다른 단점은 관리가 어려우며 시공 중에 매우 민감하다는 점입니다. 숙성기간이 필요하고 열에 취약하다는 단점 외에도 시공과 유지보수부터 관리까지 복잡한 과정이 필요하여 추가적인 주의가 요구됩니다.'

In [14]:
gemma_ai_assistant.query('비드법 보온판 1종의 장점은 무엇인가요?')

'비드법 보온판 1종의 장점은 다음과 같습니다. 1. 가볍고 용이한 시공: 비드법 보온판 1종은 무게가 가벼워서 비교적 용이하게 시공할 수 있습니다. 2. 우수한 단열성능: 높은 단열성능을 가지고 있어, 건물 내부의 온도와 습도를 안정적으로 유지시킬 수 있습니다. 3. 탁월한 난연성능: 비드법 보온판 1종은 난연성능이 뛰어나 화재 발생 시 화재의 확산을 방지하는 역할을 합니다. 4. 내구성과 유지보수비용 절감: 비드법 보온판 1종은 오랜 기간 동안 안정적인 성능을 유지하고, 추가적인 보수 및 유지보수 비용을 절감할 수 있습니다.'

In [17]:
# 다른 프롬프트 확인
gemma_ai_assistant.query('비드법 보온판이 뭐야')

'비드법 보온판은 단열재로서의 역할 뿐만 아니라 흡음재, 차음재, 단열이 되는 특징을 갖고 있어 다양한 용도로 활용됩니다. 이러한 보온판은 가정의 방음에 활용되며, 전문가의 조언에 따라 다양한 용도로 활용됩니다.'

In [15]:
gemma_ai_assistant.query('쌀풀은 무엇이며 어떤 용도로 사용되나요?')

'쌀풀은 쌀을 가공하여 제작된 재료로, 친환경적이고 접착력이 밀풀에 비해 약 1.5배 좋으며 내구성이 좋은 재료입니다. 이러한 장점을 갖고 있지만, 부착면과 내부 마감용으로 사용되며, 가벼우면서도 부드러운 질감을 가지고 있어 인테리어에 활용됩니다.'

In [16]:
# 다른 프롬프트 확인
gemma_ai_assistant.query('쌀풀이 뭐야')

'쌀풀은 쌀을 가공하여 제작된 재료로, 친환경적이고 접착력이 밀풀에 비해 약 1.5배 좋으며 내구성이 좋은 재료입니다. 이러한 장점을 갖고 있지만, 부착된 먼지와 이물질이 쉽게 제거되지 않는 경우도 있습니다. 이러한 특성을 고려하여 쌀풀을 사용할 때에는 정기적으로 청소를 하는 것이 좋습니다.'

In [18]:
# 관계없는 답변 확인
gemma_ai_assistant.query('오늘 날씨를 알려줘')

'오늘 날씨를 알려주는 방법은 여러 가지가 있습니다. 먼저, 온도와 습도를 확인하는 방법이 있습니다. 온도는 기온이 측정된 장소에서 얻을 수 있으며, 습도는 습기가 고여있는 상황을 나타내는 측정값입니다. 또한, 시간대별로 날씨가 변화하는 것을 감지하기 위해 온도와 습도 외에도 시간대별 날씨를 확인할 수도 있습니다. 또한, 기후 환경을 고려하여 햇빛 노출 시간, 오염물질 농도, 그리고 건강한 식물플랜트의 생육상황 등도 확인할 수 있습니다. 이러한 다양한 방법을 통해 오늘 날씨를 좀 더 자세히 알릴 수 있습니다.'

In [22]:
!pip install -q -U sentence_transformers
from sentence_transformers import SentenceTransformer

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [25]:
# test run

import random
rand_num = random.choices(range(len(train)), k=20)


# Embedding Vector 추출에 활용할 모델(distiluse-base-multilingual-cased-v1) 불러오기
test_model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

In [26]:
# 샘플에 대한 Cosine Similarity 산식
def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b) if norm_a != 0 and norm_b != 0 else 0

sample_scores = []
for i in rand_num:
    question, gt = train.iloc[i]['question'], train.iloc[i]['answer']
    pred = gemma_ai_assistant.query(question)

    # 생성된 답변 내용을 512 Embedding Vector로 변환
    pred_embed = test_model.encode(pred)
    gt_embed = test_model.encode(gt)

    sample_score = cosine_similarity(gt_embed, pred_embed)
    # Cosine Similarity Score가 0보다 작으면 0으로 간주
    sample_score = max(sample_score, 0)
    print('예측 : ', pred)
    print('정답 : ', gt)
    print('Cosine Similarity Score : ', sample_score)
#     print('-'*20)
    sample_scores.append(sample_score)
print('전체 샘플의 Cosine Similarity Score 평균 : ', np.mean(sample_scores))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

예측 :  KMEW 세라믹 지붕재는 일본의 기업인 KMEW에서 개발한 지붕재로, 산화티타늄 처리를 통해 내구성을 높이고 환경친화적인 소재로 제작되었습니다. 이 지붕재는 태양광 및 자외선으로부터의 보호와 함께 오염물질의 분해를 돕는 기능을 갖고 있어, 건물의 외관을 보호하고 유지보수에 도움이 되는 장점을 가지고 있습니다.
정답 :  KMEW 세라믹 지붕재는 일본의 기업인 KMEW에서 개발한 지붕재로, 산화티타늄 처리를 통해 내구성을 높이고 환경친화적인 소재로 제작되었습니다. 이 지붕재는 태양광 및 자외선으로부터의 보호와 함께 오염물질의 분해를 돕는 기능을 갖고 있어, 건물의 외관을 보호하고 유지보수에 도움이 되는 장점을 가지고 있습니다.
Cosine Similarity Score :  1.0000001


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

예측 :  실내 조명을 선택할 때에는 공간의 목적과 사용 용도에 적합한 조명을 고려해야 합니다. 주로 사용되는 종류로는 일반 조명 외에 간접 조명, 작업등, 분위기등이 있습니다. 일반 조명은 주로 전체 공간을 밝히는 데 사용되며, 간접 조명은 부드러운 분위기를 연출하거나 특정 구역을 강조할 때 사용됩니다. 작업등은 작업 공간이나 독서 공간 등에서 집중적인 조명이 필요할 때 사용되며, 분위기등은 분위기를 조성하고 강조하는 데 사용됩니다. 이러한 다양한 종류의 조명을 적절히 조합하여 공간에 적합한 조명을 선택하는 것이 중요합니다.
정답 :  실내 조명을 선택할 때에는 공간의 목적과 활용 목적에 맞춰 다양한 종류의 조명을 고려해야 합니다. 가장 기본적인 일반 조명 외에도 간접 조명, 작업등, 분위기등과 같은 다양한 조명 종류를 고려할 수 있습니다. 예를 들어 작업공간이라면 작업등을 고려하고, 실내 분위기를 부드럽게 조성하고자 한다면 간접 조명을 고려할 수 있습니다. 따라서 조명을 선택할 때에는 공간의 사용 목적과 원하는 분위기를 고려하여 종류를 선택하는 것이 중요합니다.
Cosine Similarity Score :  0.9672125


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

예측 :  PVC 샤시는 폴리염화비닐(이하 PVC)로 만들어진 샤시로, 일반 샤시보다 튼튼하고 내구성이 뛰어나며, 가벼워서 비교적 용이하게 제작할 수 있는 장점이 있습니다. 이러한 샤시는 주로 실내 및 외부에서의 사용에 적합하며, 낮은 열전도율, 내열성, 내구성이 뛰어나 화재에 대한 내성이 뛰어나다는 장점이 있습니다. 또한, PVC 재료는 유해물질(VOC)이 적고 친환경적이기 때문에 인기가 높습니다.
정답 :  PVC 샤시는 창문이나 문 등의 구조물을 만들 때 사용되는 재료로, 폴리염화비닐이라는 열가소성 플라스틱으로 제작된 샤시입니다. 이러한 PVC 샤시는 내구성과 저렴한 비용으로 유명합니다. 또한 절연성이 뛰어나고 부식에 강하며 유지보수가 쉽다는 장점이 있습니다.
Cosine Similarity Score :  0.820067
전체 샘플의 Cosine Similarity Score 평균 :  0.7152254
